### Detectar formacion de canal Bajista - Alcista

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from bokeh.models import NumeralTickFormatter, Span
from sklearn.linear_model import LinearRegression

In [2]:
df_h = pd.read_csv(r"D:\PRUEBAS\dataxh.txt", sep="\t")
tickers = [
     'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [3]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA35'] = company['Close'].rolling(35).mean()
    company.dropna(inplace=False)
    company['SMA50'] = company['Close'].rolling(50).mean()
    company.dropna(inplace=False)
    company['EMA35'] = company['Close'].ewm(span=35, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA50'] = company['Close'].ewm(span=50, adjust=False).mean()
    company.dropna(inplace=False)
    
    #company['SMA100'] = company['Close'].rolling(100).mean()
    #company.dropna(inplace=False)
    #company['SMA200'] = company['Close'].rolling(200).mean()
    #company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

### Detect Pivots/Fractals

In [4]:
ord=15
ord2=5
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]

max_idx2 = argrelextrema(df['Close'].values, np.greater, order=ord2)[0]
min_idx2 = argrelextrema(df['Close'].values, np.less, order=ord2)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

df.loc[max_idx2, 'pivotHigh2'] = df['High']+1e-3
df.loc[min_idx2, 'pivotLow2'] = df['Low']-(1e-3)
df.loc[max_idx2, 'isPivot2'] = 1
df.loc[min_idx2, 'isPivot2'] = 2

###### Canales bajistas, 
###### - Cuando el PM de 40 esta por encima del PM de 20 en direccion desendente
###### - Cuando se dan el cruce de medias moviles 35 y 50

In [5]:
#df = df.drop(columns=['cruce_medias']) # Drops column 'B'

In [6]:
# Señal: 1 = compra, -1 = venta, 0 = nada
#df['prev_SMA35'] = df['SMA35'].shift(1)
#df['prev_SMA50'] = df['SMA50'].shift(1)
df['prev_EMA35'] = df['EMA35'].shift(1)
df['prev_EMA50'] = df['EMA50'].shift(1)


#Cruce de medias
#df['cruce_medias'] = 0
#df.loc[(df['prev_SMA35'] < df['prev_SMA50']) & (df['SMA35'] > df['SMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
#df.loc[(df['prev_SMA35'] > df['prev_SMA50']) & (df['SMA35'] < df['SMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

df['cruce_medias'] = 0
df.loc[(df['prev_EMA35'] < df['prev_EMA50']) & (df['EMA35'] > df['EMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
df.loc[(df['prev_EMA35'] > df['prev_EMA50']) & (df['EMA35'] < df['EMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

In [7]:
df["cdlengulfing"] = ta.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlhammer"] = ta.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlmorningstar"] = ta.CDLMORNINGSTAR(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlpiercing"] = ta.CDLPIERCING(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlmarubozu"] = ta.CDLMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
df["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlharami"] = ta.CDLHARAMI(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlharamicross"] = ta.CDLHARAMICROSS(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlladderbottom"] = ta.CDLLADDERBOTTOM(df['Open'], df['High'], df['Low'], df['Close'])

In [8]:
#df = df.drop(columns=['indcaso']) # Drops column 'B'

In [9]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []

ispivot=0
for i, row in df.iterrows():
    #if (i==512):
    cant = 0
    cant2 = 0
    ind_sl=0
    ind_trendHL=0

    ticker = df['companyName'][i]
    #Reinicio de casos por company
    if ticker2 != ticker:
        ticker2 = ticker
        caso = 0
    dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval].copy()
            
    #if (dfpl["cruce_medias"][i]==-1):
    #    #print("posible caso a la BAJA:",i, df['companyName'][i])
    #    ispivot=1
    #    ispivot2=2
    #    if (sl<0):
    #        #print("h1")
    #        ind_sl=1
    #
    #elif (dfpl["cruce_medias"][i]==1):
    #    #print("posible caso a la ALZA:",i, df['companyName'][i])
    #    ispivot=2
    #    ispivot2=1
    #    if (sl>0):
    #        #print("h2")
    #        ind_sl=1
    ind_sl=0
    tipo=""
    if ((dfpl["cruce_medias"][i]==-1) | (dfpl["cruce_medias"][i]==1)):
        if (dfpl["cruce_medias"][i]==-1):
           tipo="BAJA" 
        else:
            tipo="ALZA"
        print("posible caso:",i, df['companyName'][i], "==>", tipo)

        #Se dio el cruce de medias, antes existio un pivot high aprox 2 dias como maximo
        fecha= dfpl['datetime'][i]
        #fecha_limite = dfpl['datetime'][i] - pd.Timedelta(days=2) #resta dias calendarios
        fecha_limite = dfpl['datetime'][i] - pd.offsets.BDay(2) #resta dias laborables
        #print("fecha:", dfpl['datetime'][i] ,", fecha_limite:", fecha_limite)
        
        #cnt1 = dfpl.query("isPivot==@ispivot and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()
        #cnt2 = dfpl.query("isPivot==@ispivot2 and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()

        cnt1=1
        cnt2=0

        indiceFinal=0
        
        #print("cantidad:", cnt)
        if (cnt1>0 and cnt2==0):
            #Obtener casos      
            #print("h00")
            if (dfpl["cruce_medias"][i]==1): #ALCISTA               
                #ALZA, velas por encima de promedios moviles
                #ultimo high por encima y ultimo low cerca a los promedios
                siguiente_H = dfpl[(dfpl.index>=i) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                if (siguiente_H.shape[0]>0):
                    if (siguiente_H.iloc[0]['High']>siguiente_H.iloc[0]['EMA35']):
                        indice = siguiente_H.index[0]
                        siguiente_L = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)
                        if (siguiente_L.shape[0]>0):
                            if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1) &  (siguiente_L.iloc[0]['Low']<siguiente_H.iloc[0]['High'])):
                                indiceFinal = siguiente_L.index[0]

            elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
                #BAJA, velas por debajo de promedios moviles
                #ultimo low por debajo y ultimo high cerca a los promedios
                #print("h01")
                siguiente_L = dfpl[(dfpl.index>=i) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)) ].head(1)
                if (siguiente_L.shape[0]>0):
                    #print("h02")
                    if (siguiente_L.iloc[0]['Low']<siguiente_L.iloc[0]['EMA35']):
                        #print("h03")
                        indice = siguiente_L.index[0]
                        #print("indice:", indice)
                        siguiente_H = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                        if (siguiente_H.shape[0]>0):
                            #print("h04:",siguiente_H.index[0])
                            if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1) & (siguiente_H.iloc[0]['High']>siguiente_L.iloc[0]['Low'])):
                                #print("h05")
                                indiceFinal = siguiente_H.index[0]

            #print ("indiceFinal:", indiceFinal)
            if (indiceFinal>0):
                #print("h0")
                ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1)].loc[:indiceFinal].tail(2)
                ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)].loc[:indiceFinal].tail(2)
                    
                if (ultimos_2H.shape[0]==2 & ultimos_2L.shape[0]==2):
                    #print("h1")
                    penultimo_valorH = ultimos_2H.iloc[0]['High']
                    ultimo_valorH = ultimos_2H.iloc[1]['High']
                    penultimo_valorL = ultimos_2L.iloc[0]['Low']
                    ultimo_valorL = ultimos_2L.iloc[1]['Low']

                    #En caso de tendencia ALCISTA
                    #tomar los 2 ultimos HH – Higher High (Maximos mas Altos)
                    #tomar los 2 ultimos HL – Higher Low (Maximos mas bajos)
                    if (dfpl["cruce_medias"][i]==1): #ALCISTA
                        #print("h2")
                        if ((ultimo_valorH>penultimo_valorH) | (ultimo_valorL>penultimo_valorL)):
                            #print("h21")
                            ind_trendHL=1

                    #En caso de tendencia BAJISTA
                    #tomar los 2 ultimos LH – Lower High (Minimos mas Altos)
                    #tomar los 2 ultimos LL – Lower Low (Minimos mas bajos)
                    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
                        #print("h3")
                        if ((ultimo_valorH<penultimo_valorH) | (ultimo_valorL<penultimo_valorL)):
                            #print("h31")
                            ind_trendHL=1

            #print("indiceFinal:",indiceFinal,",ind_trendHL:",ind_trendHL)

            if (indiceFinal>0):
                #Inicio determinar slope que indica tendencia al ALZA o a la BAJA
                medias35 = dfpl.loc[i:indiceFinal].EMA35.values
                idxmedias35 = dfpl.loc[i:indiceFinal].EMA35.index
                medias50 = dfpl.loc[i:indiceFinal].EMA50.values
                idxmedias50 = dfpl.loc[i:indiceFinal].EMA50.index
                if ((len(medias35)>=2) and  (len(medias50)>=2)):
                    sl35, interc35, r_value35, _, _ = stats.linregress(idxmedias35,medias35)
                    sl50, interc50, r_value50, _, _ = stats.linregress(idxmedias50,medias50)
                #Fin slope
        
                if (dfpl["cruce_medias"][i]==1): #ALCISTA
                    #Revision de pendiente
                    if (sl35>0 and sl50>0 #Pendiente positiva
                       #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
                       #and interc35>interc50 #La línea 35 está por encima de la línea 50
                       ):
                        ind_sl=1
                elif (dfpl["cruce_medias"][i]==-1): #BAJISTA 
                    #Revision de pendiente
                    if (sl35<0 and sl50<0 #Pendiente negativa
                        #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
                        #and interc50>interc35 #La línea 50 está por encima de la línea 35
                       ):
                        ind_sl=1
    
            #print("sl35:",sl35,"sl50:",sl50)
            #print("interc35:",interc35,"interc50:",interc50, ", ind_sl:", ind_sl)

            
            
            if ((indiceFinal>0) and (ind_trendHL>0) and (ind_sl>0)):
                #print("h4")
                #INSERT CASO
                caso=caso+1
                dfpl.loc[i,'ind_posicion']=0
               

                #Final de caso es siguiente Pivot
                if (dfpl["cruce_medias"][i]==1): #ALCISTA
                    dfpl.loc[indiceFinal,'isBreakOutIni']=1
                    k=0
                    while (k<=3):
                        cnt2 = dfpl.query("index>@indiceFinal and isPivot==1").shape[0]
                        if cnt2>0:        
                            id2 = dfpl.query("index>@indiceFinal and isPivot==1").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = 1
                            k=4
                        else:
                            idfinal = dfpl.index[-1] 
                            idfinal2 = idfinal+25
                            if idfinal2 in df[df['companyName']==ticker].index:
                                dfpl2 = df[(df.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                            else:                  
                                dfpl['isBreakOutFinal'] = np.nan
                            k=k+1
                           
                
                        
                elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
                    dfpl.loc[indiceFinal,'isBreakOutIni']=-1
                    k=0
                    while (k<=3):
                        cnt2 = dfpl.query("index>@indiceFinal and isPivot==2").shape[0]
                        if cnt2>0:        
                            id2 = dfpl.query("index>@indiceFinal and isPivot==2").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = -1
                            k=4
                        else:
                            idfinal = dfpl.index[-1] 
                            idfinal2 = idfinal+25
                            if idfinal2 in df[df['companyName']==ticker].index:
                                dfpl2 = df[(df.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                            else:                  
                                dfpl['isBreakOutFinal'] = np.nan
    
                            k=k+1

                
                if (df["cruce_medias"][i]==-1):                    
                    df.loc[i,'ind_posicion']=-1                    
                    df.loc[indiceFinal,'isBreakOutIni']=-1
                    df.loc[id2,'isBreakOutFinal']=-1
                elif (df["cruce_medias"][i]==1):                        
                    df.loc[i,'ind_posicion']=1                   
                    df.loc[indiceFinal,'isBreakOutIni']=1
                    df.loc[id2,'isBreakOutFinal']=1

                #print("h5")
                dfpl["caso"] = caso
                print(i, df['companyName'][i], ", cruce medias:",df["cruce_medias"][i],", slope35:", sl35, ", slope50:", sl50)
                if len(df_casos)<1:
                    df_casos = dfpl
                else:
                    df_casos = pd.concat([df_casos, dfpl],ignore_index=False)

posible caso: 22 SPY ==> BAJA
22 SPY , cruce medias: -1 , slope35: -0.07934301034719624 , slope50: -0.06400623303169807
posible caso: 48 SPY ==> ALZA
48 SPY , cruce medias: 1 , slope35: 0.24577412254642428 , slope50: 0.1989712377727817
posible caso: 212 SPY ==> BAJA
212 SPY , cruce medias: -1 , slope35: -0.18260016581871708 , slope50: -0.14115528949812053
posible caso: 340 SPY ==> ALZA
340 SPY , cruce medias: 1 , slope35: 0.17555931399589864 , slope50: 0.16701117424564785
posible caso: 394 SPY ==> BAJA
posible caso: 395 SPY ==> ALZA
395 SPY , cruce medias: 1 , slope35: 0.04149601880466476 , slope50: 0.03303738893741972
posible caso: 428 SPY ==> BAJA
428 SPY , cruce medias: -1 , slope35: -0.1544552233125668 , slope50: -0.12313649729010304
posible caso: 542 SPY ==> ALZA
posible caso: 596 SPY ==> BAJA
posible caso: 664 SPY ==> ALZA
posible caso: 949 SPY ==> BAJA
949 SPY , cruce medias: -1 , slope35: -0.11455394346089216 , slope50: -0.09734176410930492
posible caso: 978 SPY ==> ALZA
978 SP

In [10]:
#df_casos[['companyName', 'caso']].unique()
#resultado = df_casos.groupby(['companyName', 'caso'])['Close'].count().reset_index()
#resultado[resultado['companyName']=="META"].shape[0]

In [ ]:
#CODIGO DE REVISION CARLOS
#dfpl = df.iloc[500:600]
dfpl = df[df["companyName"]=="SPY"]

dfpl["Datetime_str"] = dfpl["datetime"].astype(str)

p = figure(width=2500, height=500,
        title="Canal Bajista - Alcista",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"),("datetime", "@Datetime_str"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close"),
                    ("EMA35", "@EMA35"),
                    ("EMA50", "@EMA50"),
                    ("SMA35", "@SMA35"),
                    ("SMA50", "@EMA50"),
                    ("cdlengulfing","@cdlengulfing"), 
                    ("cdlhammer","@cdlhammer"), 
                    ("cdlmorningstar","@cdlmorningstar"), 
                    ("cdlpiercing","@cdlpiercing"), 
                    ("cdlclosingmarubozu","@cdlclosingmarubozu"), 
                    ("cdlmarubozu","@cdlmarubozu"), 
                    ("cdl3whitesoldiers","@cdl3whitesoldiers"), 
                    ("cdlharami","@cdlharami"), 
                    ("cdlharamicross","@cdlharamicross"), 
                    ("cdlinvertdhammer","@cdlinvertdhammer"), 
                    ("cdlladderbottom","@cdlladderbottom")]
        #tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.line(
    x="index", 
    y="SMA35", 
    color="white",
    legend_label="SMA35",
    source=dfpl)


p.line(
    x="index", 
    y="SMA50", 
    color="blue",
    legend_label="SMA50",
    source=dfpl)


p.line(
    x="index", 
    y="EMA35", 
    color="black",
    legend_label="EMA35",
    source=dfpl)


p.line(
    x="index", 
    y="EMA50", 
    color="red",
    legend_label="EMA50",
    source=dfpl)

#p.line(
#    x="index", 
#    y="SMA100", 
#    color="green",
#    legend_label="SMA100",
#    source=dfpl)
#
#p.line(
#    x="index", 
#    y="SMA200", 
#    color="purple",
#    legend_label="SMA200",
#    source=dfpl)



p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="inicio1", source=dfpl[(dfpl.isBreakOutIni==1)])

p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="red", alpha=0.5, legend_label="inicio2", source=dfpl[(dfpl.isBreakOutIni==-1)])


p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="black", alpha=0.5, legend_label="fin1", source=dfpl[(dfpl.isBreakOutFinal==1)])

p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="brown", alpha=0.5, legend_label="fin2", source=dfpl[(dfpl.isBreakOutFinal==-1)])




p.scatter(x="index", y="pivotHigh", marker="circle", size=5,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista1", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=5,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista1", source=dfpl[(dfpl.isPivot==2)])


p.scatter(x="index", y="pivotHigh2", marker="diamond", size=8,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista2", source=dfpl[(dfpl.isPivot2==1) & (dfpl.isPivot.isna()) ])

p.scatter(x="index", y="pivotLow2", marker="diamond", size=8,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista2", source=dfpl[(dfpl.isPivot2==2) & (dfpl.isPivot.isna())])

p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Precio"
p.legend.location="top_left"
p.legend.click_policy="hide"

#Render linea vertical
alza = (dfpl[(dfpl.ind_posicion==1)].index).tolist()
baja = (dfpl[(dfpl.ind_posicion==-1)].index).tolist()
#vline=Span(source=inicio,dimension='height', line_color='blue',line_width=0.8, line_dash_offset= 0, line_dash='dashed', name="hola esto es una prueba", level='annotation', tags= ['square'])
#p.renderers.extend([vline])

p.vspan(
    x=alza,
    line_color="blue"
)

p.vspan(
    x=baja,
    line_color="red"
)

show(p)

In [11]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
#path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
path = r'D:\PRUEBAS\cba_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

#### BACKTESTING
###### Usando paquete backtesting.py

In [12]:
from backtesting import Backtest, Strategy

C:\Users\LINDER\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [13]:
df_backTesting = df.copy()
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
df_backTesting.set_index('datetime', inplace=True)

In [14]:
class StrategyCanalBajistaAlcista(Strategy):
    def init(self):
        # Registramos las señales como 'indicadores' para que Backtest las procese correctamente
        self.entry_signal = self.I(lambda x: x, self.data.isBreakOutIni)
        self.exit_signal = self.I(lambda x: x, self.data.isBreakOutFinal)

    def next(self):
        # Verificar si hay posición abierta
        in_position = self.position.is_long or self.position.is_short

        # Salida: si la señal coincide con el tipo de posición
        if in_position:
            if (self.position.is_long and self.exit_signal[-1] == 1) or \
               (self.position.is_short and self.exit_signal[-1] == -1):
                self.position.close()
                return

        # Entrada: abrir solo si no hay posición previa
        if not in_position:
            if self.entry_signal[-1] == 1:
                self.buy()
            elif self.entry_signal[-1] == -1:
                self.sell()


In [15]:
def run_backtests(df_backTesting, initial_cash=10_000, commission=0.0):
    # Asegurar índice datetime y orden cronológico
    df_backTesting = df_backTesting.copy()
    if not pd.api.types.is_datetime64_any_dtype(df_backTesting.index):
        if 'Date' in df_backTesting.columns:
            df_backTesting.index = pd.to_datetime(df_backTesting['Date'])
        else:
            df_backTesting.index = pd.to_datetime(df_backTesting.index)
    df_backTesting.sort_index(inplace=True)

    all_stats = []
    all_trades = []

    for ticker, data in df_backTesting.groupby('companyName'):
        # Verificar que haya suficientes datos
        if data.shape[0] < 2:
            continue

        bt = Backtest(
            data,
            StrategyCanalBajistaAlcista,
            cash=initial_cash,
            commission=commission
        )
        stats = bt.run()

        # Estadísticas por ticker
        stats_dict = {
            'Ticker': ticker,
            'Start': stats['Start'],
            'End': stats['End'],
            'Return [%]': stats['Return [%]'],
            'CAGR [%]': stats['Return (Ann.) [%]'],
            'Sharpe Ratio': stats['Sharpe Ratio'],
            'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
            'Win Rate [%]': stats['Win Rate [%]'],
            '# Trades': stats['# Trades'],
            'Profit Factor': stats['Profit Factor'],
            'Expectancy [%]': stats['Expectancy [%]'],
            'Avg. Trade [%]': stats['Avg. Trade [%]'],
        }
        all_stats.append(stats_dict)

        # Detalle de operaciones
        trades_df = pd.DataFrame(stats['_trades'])
        if not trades_df.empty:
            trades_df['Ticker'] = ticker
            all_trades.append(trades_df)

    df_stats = pd.DataFrame(all_stats)
    df_trades = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
    return df_stats, df_trades


In [16]:
df_stats, df_trades = run_backtests(df_backTesting, commission=0.001)

Backtest.run:   0%|          | 0/3400 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3333 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3473 [00:00<?, ?bar/s]

In [17]:
df_trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_λ(isBreakOu…),Exit_λ(isBreakOu…),Ticker
0,57,668,762,174.240005,191.914993,None,None,1007.474304,0.101440,2023-11-03 13:30:00+00:00,2023-11-22 17:30:00+00:00,19 days 04:00:00,None,NaN,NaN,AAPL
1,57,1064,1112,190.869995,190.639999,None,None,-13.109756,-0.001205,2024-01-29 15:30:00+00:00,2024-02-07 14:30:00+00:00,8 days 23:00:00,None,NaN,NaN,AAPL
2,63,1326,1356,172.100098,173.000000,None,None,56.693848,0.005229,2024-03-21 17:30:00+00:00,2024-03-27 19:30:00+00:00,6 days 02:00:00,None,NaN,NaN,AAPL
3,-66,1473,1524,166.270004,172.384995,None,None,-403.589355,-0.036777,2024-04-22 17:30:00+00:00,2024-05-01 19:30:00+00:00,9 days 02:00:00,None,NaN,NaN,AAPL
4,-46,1907,1920,224.639999,219.489899,None,None,236.904633,0.022926,2024-07-23 14:30:00+00:00,2024-07-25 13:30:00+00:00,1 days 23:00:00,None,NaN,NaN,AAPL
5,-48,2123,2141,221.520004,220.837997,None,None,32.736328,0.003079,2024-09-05 13:30:00+00:00,2024-09-09 17:30:00+00:00,4 days 04:00:00,None,NaN,NaN,AAPL
6,47,2214,2248,229.179993,231.205002,None,None,95.175430,0.008836,2024-09-24 13:30:00+00:00,2024-09-30 19:30:00+00:00,6 days 06:00:00,None,NaN,NaN,AAPL
7,-48,2269,2283,223.710007,222.563995,None,None,55.008545,0.005123,2024-10-03 19:30:00+00:00,2024-10-07 19:30:00+00:00,4 days 00:00:00,None,NaN,NaN,AAPL
8,47,2298,2319,227.779999,233.589996,None,None,273.069885,0.025507,2024-10-10 13:30:00+00:00,2024-10-15 13:30:00+00:00,5 days 00:00:00,None,NaN,NaN,AAPL
9,-50,2431,2454,221.235504,222.779999,None,None,-77.224731,-0.006981,2024-11-06 14:30:00+00:00,2024-11-11 16:30:00+00:00,5 days 02:00:00,None,NaN,NaN,AAPL


In [38]:
df_trades[df_trades['Ticker']=='SPY']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_λ(isBreakOu…),Exit_λ(isBreakOu…),Ticker
38,-22,222,290,451.554993,433.369995,None,None,400.069946,0.040272,2023-08-04 15:30:00+00:00,2023-08-18 13:30:00+00:00,13 days 22:00:00,None,NaN,NaN,SPY
39,23,381,400,443.109985,448.429993,None,None,122.360168,0.012006,2023-09-07 13:30:00+00:00,2023-09-11 18:30:00+00:00,4 days 05:00:00,None,NaN,NaN,SPY
40,23,407,422,446.859985,450.309998,None,None,79.350281,0.007721,2023-09-12 18:30:00+00:00,2023-09-14 19:30:00+00:00,2 days 01:00:00,None,NaN,NaN,SPY
41,-23,446,482,443.274994,424.200012,None,None,438.724579,0.043032,2023-09-20 15:30:00+00:00,2023-09-27 16:30:00+00:00,7 days 01:00:00,None,NaN,NaN,SPY
42,-23,971,988,473.660004,473.049988,None,None,14.030365,0.001288,2024-01-08 20:30:00+00:00,2024-01-11 16:30:00+00:00,2 days 20:00:00,None,NaN,NaN,SPY
43,-21,1394,1467,517.789978,495.459991,None,None,468.929718,0.043126,2024-04-05 15:30:00+00:00,2024-04-19 18:30:00+00:00,14 days 03:00:00,None,NaN,NaN,SPY
44,-21,1672,1707,529.020020,533.179993,None,None,-87.359436,-0.007864,2024-06-03 13:30:00+00:00,2024-06-10 13:30:00+00:00,7 days 00:00:00,None,NaN,NaN,SPY
45,-20,1907,1926,556.020020,543.130005,None,None,257.800293,0.023183,2024-07-23 14:30:00+00:00,2024-07-25 19:30:00+00:00,2 days 05:00:00,None,NaN,NaN,SPY
46,20,2065,2067,557.039978,559.530029,None,None,49.801025,0.004470,2024-08-22 18:30:00+00:00,2024-08-23 13:30:00+00:00,0 days 19:00:00,None,NaN,NaN,SPY
47,19,2862,2912,600.609070,612.109985,None,None,218.517395,0.019149,2025-02-07 19:30:00+00:00,2025-02-19 20:30:00+00:00,12 days 01:00:00,None,NaN,NaN,SPY


In [42]:
df_casos[(df_casos['companyName']=='SPY') & ((df_casos['isBreakOutIni']==1) | (df_casos['isBreakOutFinal']==1)) ]

,Close,High,Low,Open,Volume,datetime,companyName,SMA35,SMA50,EMA35,...,cdlmarubozu,cdl3whitesoldiers,cdlharami,cdlharamicross,cdlinvertdhammer,cdlladderbottom,ind_posicion,isBreakOutIni,isBreakOutFinal,caso
74,437.350006,437.850006,437.059998,437.730011,9701525,2023-07-06 14:30:00+00:00,SPY,440.002088,438.025425,440.214176,...,0,0,0,0,0,0,NaN,1.0,-1.0,2
136,455.820007,456.339996,455.010010,455.010010,13482023,2023-07-19 13:30:00+00:00,SPY,449.985440,447.151818,449.892159,...,0,0,0,0,0,0,NaN,NaN,1.0,2
381,443.230011,444.399994,442.804993,443.109985,12884460,2023-09-07 13:30:00+00:00,SPY,449.574809,448.297410,447.674797,...,0,0,0,0,0,0,NaN,1.0,NaN,4
400,448.589996,448.700012,448.170013,448.429993,6566213,2023-09-11 18:30:00+00:00,SPY,446.884119,448.193675,446.932618,...,0,0,0,0,0,0,NaN,NaN,1.0,4
381,443.230011,444.399994,442.804993,443.109985,12884460,2023-09-07 13:30:00+00:00,SPY,449.574809,448.297410,447.674797,...,0,0,0,0,0,0,NaN,1.0,NaN,5
400,448.589996,448.700012,448.170013,448.429993,6566213,2023-09-11 18:30:00+00:00,SPY,446.884119,448.193675,446.932618,...,0,0,0,0,0,0,NaN,NaN,1.0,5
407,445.630005,446.980011,445.390015,446.859985,9169512,2023-09-12 18:30:00+00:00,SPY,446.261238,447.556110,446.974801,...,0,0,0,0,0,0,NaN,1.0,NaN,5
422,450.399994,450.649994,450.112915,450.309998,15177636,2023-09-14 19:30:00+00:00,SPY,447.476887,446.842945,447.893030,...,0,0,0,0,0,0,NaN,NaN,1.0,5
381,443.230011,444.399994,442.804993,443.109985,12884460,2023-09-07 13:30:00+00:00,SPY,449.574809,448.297410,447.674797,...,0,0,0,0,0,0,NaN,1.0,NaN,6
400,448.589996,448.700012,448.170013,448.429993,6566213,2023-09-11 18:30:00+00:00,SPY,446.884119,448.193675,446.932618,...,0,0,0,0,0,0,NaN,NaN,1.0,6


In [30]:
ini=(df_casos[df_casos['isBreakOutIni']==1]).copy()
df_casos_ini=ini.rename(columns={'Open':'EntryPrice','datetime':'EntryTime','companyName':'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]

In [31]:
df_casos_ini

,EntryPrice,Ticker,caso,EntryTime
74,437.730011,SPY,2,2023-07-06 14:30:00+00:00
381,443.109985,SPY,4,2023-09-07 13:30:00+00:00
381,443.109985,SPY,5,2023-09-07 13:30:00+00:00
407,446.859985,SPY,5,2023-09-12 18:30:00+00:00
381,443.109985,SPY,6,2023-09-07 13:30:00+00:00
407,446.859985,SPY,6,2023-09-12 18:30:00+00:00
988,473.049988,SPY,8,2024-01-11 16:30:00+00:00
2065,557.039978,SPY,12,2024-08-22 18:30:00+00:00
2862,600.609070,SPY,13,2025-02-07 19:30:00+00:00
3538,600.750000,SPY,16,2025-06-16 19:00:00+00:00


In [44]:
final=(df_casos[df_casos['isBreakOutFinal']==1]).copy()
df_casos_final=final.rename(columns={'Open':'ExitPrice','companyName':'Ticker','datetime':'ExitTime'})[['ExitPrice','Ticker','ExitTime','caso']]

In [45]:
df_casos_final

,ExitPrice,Ticker,ExitTime,caso
136,455.010010,SPY,2023-07-19 13:30:00+00:00,2
400,448.429993,SPY,2023-09-11 18:30:00+00:00,4
400,448.429993,SPY,2023-09-11 18:30:00+00:00,5
422,450.309998,SPY,2023-09-14 19:30:00+00:00,5
400,448.429993,SPY,2023-09-11 18:30:00+00:00,6
422,450.309998,SPY,2023-09-14 19:30:00+00:00,6
997,476.640015,SPY,2024-01-12 18:30:00+00:00,8
2067,559.530029,SPY,2024-08-23 13:30:00+00:00,12
2912,612.109985,SPY,2025-02-19 20:30:00+00:00,13
3780,315.510010,META,2023-08-07 19:30:00+00:00,2


In [46]:
df_merge_alcista=df_casos_ini.merge(df_casos_final,on=['Ticker','caso'], how='outer')

In [47]:
df_merge_alcista

,EntryPrice,Ticker,caso,EntryTime,ExitPrice,ExitTime
0,190.365005,AAPL,2,2023-07-14 15:30:00+00:00,194.649994,2023-07-19 18:30:00+00:00
1,174.240005,AAPL,3,2023-11-03 13:30:00+00:00,191.914993,2023-11-22 17:30:00+00:00
2,190.869995,AAPL,4,2024-01-29 15:30:00+00:00,190.639999,2024-02-07 14:30:00+00:00
3,190.869995,AAPL,5,2024-01-29 15:30:00+00:00,190.639999,2024-02-07 14:30:00+00:00
4,172.100098,AAPL,6,2024-03-21 17:30:00+00:00,173.000000,2024-03-27 19:30:00+00:00
5,172.740005,AAPL,8,2024-04-30 19:30:00+00:00,185.360001,2024-05-03 17:30:00+00:00
6,229.179993,AAPL,11,2024-09-24 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
7,229.179993,AAPL,12,2024-09-24 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
8,227.779999,AAPL,13,2024-10-10 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
9,227.779999,AAPL,13,2024-10-10 13:30:00+00:00,233.589996,2024-10-15 13:30:00+00:00


In [48]:
df_casos_ini['seq']   = df_casos_ini.groupby(['Ticker','caso']).cumcount() + 1
df_casos_final['seq'] = df_casos_final.groupby(['Ticker','caso']).cumcount() + 1

In [49]:
# 3) Haz el merge incluyendo esa 'seq'
df_merge = (df_casos_ini
    .merge(df_casos_final, on=['Ticker','caso','seq'], how='outer')
    .drop(columns='seq')  # si ya no la necesitas
)

In [50]:
df_merge

,EntryPrice,Ticker,caso,EntryTime,ExitPrice,ExitTime
0,190.365005,AAPL,2,2023-07-14 15:30:00+00:00,194.649994,2023-07-19 18:30:00+00:00
1,174.240005,AAPL,3,2023-11-03 13:30:00+00:00,191.914993,2023-11-22 17:30:00+00:00
2,190.869995,AAPL,4,2024-01-29 15:30:00+00:00,190.639999,2024-02-07 14:30:00+00:00
3,190.869995,AAPL,5,2024-01-29 15:30:00+00:00,190.639999,2024-02-07 14:30:00+00:00
4,172.100098,AAPL,6,2024-03-21 17:30:00+00:00,173.000000,2024-03-27 19:30:00+00:00
5,172.740005,AAPL,8,2024-04-30 19:30:00+00:00,185.360001,2024-05-03 17:30:00+00:00
6,229.179993,AAPL,11,2024-09-24 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
7,229.179993,AAPL,12,2024-09-24 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
8,227.779999,AAPL,13,2024-10-10 13:30:00+00:00,231.205002,2024-09-30 19:30:00+00:00
9,NaN,AAPL,13,NaT,233.589996,2024-10-15 13:30:00+00:00
